In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df1=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Data Collection/Journée Entreprise (réponses) - Réponses au formulaire 1 (1).csv")
df2=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Data Collection/Journée Entreprise (réponses) - Réponses au formulaire 1.csv")
df3=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Data Collection/Competency Based Curriculum 4DS4 - Réponses au formulaire 1.csv")

In [ ]:
display("4DS6",df1
        ,"4DS8",df2
        ,"4DS4",df3)

In [ ]:
dataP=pd.concat([df1,df2,df3])
dataP

In [ ]:
dataP.drop('Score', axis=1, inplace=True)

In [ ]:
dataP

In [ ]:
dataP.shape

In [ ]:
dataP.to_csv("data2.csv",index=True)

## web scraping

In [1]:
pip install cloudscraper

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests
from bs4 import BeautifulSoup
import cloudscraper
import random


In [21]:
def extract(page):
    headers={'User-Agent':  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    #scraper = cloudscraper.create_scraper(delay=10,   browser={'custom': 'ScraperBot/1.0',})
    url=f'https://www.indeed.com/jobs?q=data+scientist&start={page}'
    r= requests.get(url,headers)
    return r.status_code
    #soup =BeautifulSoup(r.content,'html.parser')
    #return soup

In [22]:
print(extract(20))

403


In [ ]:
c=extract(30)
c

In [ ]:
def transform(soup):
    divs=soup.find_all('div',class_='cardOutline tapItem ')
    return len(divs)

In [ ]:
print(transform(c))

### scraping with selenium

In [23]:
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from time import time
from time import sleep
import pandas as pd
from random import randint

In [24]:
website = """
#########################################
#           WEBSITE: INDEED.FR          #
######################################### 
"""
print(website)


#########################################
#           WEBSITE: INDEED.FR          #
######################################### 



In [25]:
start_time = datetime.now()
print('Crawl starting time : {}' .format(start_time.time()))
print()
job_list = ["data+analyst", "data+scientist", "business+analyst"]

application_links = []
job_titles = []
company_names = []
job_locations = []
application_types = []
publication_dates = []
scraping_dates = []

Crawl starting time : 00:30:11.386225



In [44]:
for job in job_list:

    opts = webdriver.FirefoxOptions()
    opts.headless = True
    profile = webdriver.FirefoxProfile('./firefoxdriver')
    profile = webdriver.FirefoxProfile()
    profile.set_preference("general.useragent.override", "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)")
    driver = webdriver.Firefox(profile, executable_path="venv/bin/geckodriver", options=opts) 

    driver.get(
            "https://www.indeed.fr/jobs?q=" + job + "&l=Paris+%2875%29&jt=fulltime&limit=50&radius=25&start=0"
        )

    sleep(randint(7,10))
    print('Collecting data for "{}"...' .format(job))
    # First, get the number of jobs available
    job_number = driver.find_element_by_xpath("//div[@id='searchCountPages']").text
    # Calculating number of pages to be crawled (number of jobs available - number of jobs per page (here, 30))
    job_number = job_number.split(" ", 4)
    job_number = int(job_number[3])
    print("- Number of open positions : {}" .format(job_number))
    exact_page_nb = job_number / 50
    print("- Exact number of pages to be crawled : {}" .format(exact_page_nb))
    min_page_nb = job_number // 50
    print("- Minimum number of pages to be crawled : {}" .format(min_page_nb))

    if exact_page_nb > min_page_nb:
        page_nb = (min_page_nb) * 50
        pages = [str(i) for i in range(0, page_nb, 50)]
    elif exact_page_nb == min_page_nb:
        page_nb = (min_page_nb - 1) * 50
        pages = [str(i) for i in range(0, page_nb, 50)]

    for page in pages:
        driver.get(
            "https://www.indeed.fr/jobs?q=" + job + "&l=Paris+%2875%29&jt=fulltime&limit=50&radius=25&start=" + page
        )

        sleep(randint(5, 12))

        # Locating job container
        all_cards = driver.find_elements_by_xpath("//div[@class='jobsearch-SerpJobCard unifiedRow row result clickcard']")

        for card in all_cards:

            # Collecting job link
            application_link = card.find_elements_by_css_selector('a')
            if not application_link:
                application_link = "Unknown"
            else:
                application_link = application_link[0].get_attribute('href')
            application_links.append(application_link)

            # Collecting job title
            job_title = card.find_elements_by_css_selector('a')
            if not job_title:
                job_title = "Unknown"
            else:
                job_title = job_title[0].text
            job_titles.append(job_title)

            # Collecting company name
            company_name = card.find_elements_by_css_selector('div.sjcl div span.company')
            if not company_name:
                company_name = "Unknown"
            else:
                company_name = company_name[0].text
            company_names.append(company_name)
            
            # Collecting job location
            job_location = card.find_elements_by_css_selector('.location.accessible-contrast-color-location')
            if not job_location:
                job_location = "Unknown"
            else:
                job_location = job_location[0].text
            job_locations.append(job_location)
            
            # Collecting application type (easy apply)
            application_type = card.find_elements_by_css_selector('.jobCardShelfContainer')
            if not application_type:
                application_type = "company's website"
            else:
                application_type = application_type[0].text
            application_types.append(application_type) 
            
            # Collecting publication date
            publication_date = card.find_elements_by_css_selector('span.date')
            if not publication_date:
                publication_date = "il y a 40 minutes"
            else:
                publication_date = publication_date[0].text
            publication_dates.append(publication_date)
            
            # Collecting generated scraping time 
            scraping_dates.append(datetime.now())

    print('Crawling status for "{}" : Done' .format(job))
    print()

    driver.quit()

print('Crawling time : {}' .format(datetime.now() - start_time))
print('Dataframe successfuly created and exported')

<ipython-input-44-25a77ea31137>:4: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  opts.headless = True
<ipython-input-44-25a77ea31137>:5: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = webdriver.FirefoxProfile('./firefoxdriver')


FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: './firefoxdriver'

In [ ]:
driver = webdriver.Firefox()

In [32]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


# Data Collection


In [1]:
import numpy as np
import pandas as pd

In [17]:
df1=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/BI_DATA.csv")
df2=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/Cloud-Data.csv")
df3=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/CloudTwinBi-Data.csv")
df4=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataScience&BI-Data.csv")
df5=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataScience-Data.csv")
df6=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/Iot.csv")
df7=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/NetworkSecurity-Data.csv")
df8=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/PROFILE SCRAPPING ESPRIT.csv")
df9=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/PROFILE SCRAPPING IOT.csv")
df10=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/PROFILE SCRAPPING WEB DEV.csv")
df11=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/result-3.csv")
df12=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/result-6.csv")
df13=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/SoftwareEngineer-Data.csv")
df14=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/TouslesProfils-Data.csv")
df15=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/TWIN-result-1.csv")

In [5]:
display(df1,df2,df3)

,error,baseUrl,timestamp,linkedinProfile,email,headline,location,imgUrl,firstName,lastName,...,school2,schoolUrl2,schoolDegree2,schoolDateRange2,jobDescription2,schoolDescription,connectedOn,birthday,mail,schoolDescription2
0,Not a LinkedIn Profile URL,profileUrl,2023-02-16T22:04:31.406Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,https://www.linkedin.com/in/hamdi-ben-zaghden-...,2023-02-16T22:05:12.029Z,https://www.linkedin.com/in/hamdi-ben-zaghden-...,NaN,ERP - Business Intelligence Engineering Studen...,"Tunis, Tunisia",https://media.licdn.com/dms/image/D4E03AQHz0Jk...,Hamdi,Ben Zaghden,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,https://www.linkedin.com/in/essid-amine-912a86240,2023-02-16T22:05:52.374Z,https://www.linkedin.com/in/essid-amine-912a86...,NaN,Ingénieur Business Intelligence,500+ connections,https://media.licdn.com/dms/image/D4D03AQGxWc1...,Essid,Amine,...,Institut Supérieur des Arts Multimédia de la M...,https://www.linkedin.com/company/12612850/,"Licence, Communication multimédia",Sep 2017 - Jul 2020,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,https://www.linkedin.com/in/hazem-baccouch-a66...,2023-02-16T22:06:33.285Z,https://www.linkedin.com/in/hazem-baccouch-a66...,NaN,Ingénieur en informatique spécialité Business ...,"Ben Arous, Tunisia",https://media.licdn.com/dms/image/C4E03AQFPwbz...,Hazem,Baccouch,...,NaN,NaN,NaN,NaN,Skills: Ingénierie,Skills: Framework Spring · Big data · RStudio ...,NaN,NaN,NaN,NaN
4,NaN,https://www.linkedin.com/in/youssef-dor%C3%A2a...,2023-02-16T22:07:13.234Z,https://www.linkedin.com/in/youssef-dor%C3%A2a...,NaN,Business Intelligence Student,"Ben Arous, Tunisia",https://media.licdn.com/dms/image/C4E03AQFTqqZ...,Youssef,Dorâa,...,NaN,NaN,NaN,NaN,NaN,Skills: Microsoft Power BI,NaN,NaN,NaN,NaN
5,NaN,https://www.linkedin.com/in/zaineb-baccar-3906...,2023-02-16T22:07:52.620Z,https://www.linkedin.com/in/zaineb-baccar-3906...,NaN,Business Intelligence engineering student at E...,"Tunis, Tunisia",https://media.licdn.com/dms/image/D4D03AQE9IGX...,Zaineb,Baccar,...,Ecole Supérieure d'Economie Numérique Manouba,https://www.linkedin.com/company/18569041/,"Licence en commerce électronique ECOM, Commerc...",Sep 2017 - Jun 2020,NaN,NaN,September 22,September 22,NaN,NaN
6,NaN,https://www.linkedin.com/in/eya-goutet-2a08a0243,2023-02-16T22:08:39.624Z,https://www.linkedin.com/in/eya-goutet-2a08a0243/,NaN,Business intelligence student,"Tunis, Tunis, Tunisia",https://media.licdn.com/dms/image/D4D03AQHxy-q...,Eya,Goutet,...,NaN,NaN,NaN,NaN,NaN,NaN,August 23,August 23,NaN,NaN
7,NaN,https://www.linkedin.com/in/wagine-hasni-17125...,2023-02-16T22:09:17.896Z,https://www.linkedin.com/in/wagine-hasni-17125...,NaN,Business Intelligence engineer,"Ariana, Tunisia",https://media.licdn.com/dms/image/D4D03AQEW556...,Wagine,HASNI,...,Ecole Supérieure Privée d'Ingénierie et de Tec...,https://www.linkedin.com/company/15134632/,"Diplôme d'ingénieur, Business Intelligence",2018 - 2023,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,https://www.linkedin.com/in/kaouther-zargouni-...,2023-02-16T22:10:00.567Z,https://www.linkedin.com/in/kaouther-zargouni-...,kaoutherzargouni@gmail.com,Élève Ingénieur en Business Intelligence - ESPRIT,"Délégation Oued Ellil, Manouba, Tunisia",https://media.licdn.com/dms/image/C4E03AQFsBkg...,Kaouther,Zargouni,...,Esprit School of Business,https://www.linkedin.com/company/84964942/,"M1, Business Analytics",2020 - 2021,NaN,NaN,NaN,NaN,kaoutherzargouni@gmail.com,NaN
9,NaN,https://www.linkedin.com/in/ameni-sbei-409a1b177,2023-02-16T22:10:41.750Z,https://www.linkedin.com/in/ameni-sbei-409a1b177/,NaN,Business intelligence engineering student at E...,"Tunis, Tunis, Tunisia",https://media.licdn.com/dms/image/C4D03AQGZxlK...,Ameni,Sbei,...,ISTIC Tunisie,NaN,"Licence Fondamentale sciences informatique , I...",2017 - 2020,Création d'une plateforme BI pour le suivi de ...,NaN,NaN,NaN,NaN,NaN


,error,baseUrl,timestamp,linkedinProfile,email,description,headline,location,imgUrl,firstName,...,schoolDegree2,connectedOn,birthday,jobDescription2,schoolDescription,website,jobLocation2,jobDescription,mail,schoolDescription2
0,Not a LinkedIn Profile URL,profileUrl,2023-02-18T16:13:09.832Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,https://www.linkedin.com/in/mohamed-achraf-cho...,2023-02-18T16:13:49.124Z,https://www.linkedin.com/in/mohamed-achraf-cho...,NaN,"AWS Cloud , Microsoft Azure , Cloud Computing ...",Software Engineer Student at ESPRIT Tunisia\n|...,"Tunis, Tunisia",https://media.licdn.com/dms/image/C4E03AQFPmwR...,Mohamed Achraf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,https://www.linkedin.com/in/aya-hammami,2023-02-18T16:14:29.963Z,https://www.linkedin.com/in/aya-hammami/,NaN,Living in the cloud !,Ops & Cloud Engineer at Chanel,Tunisia,https://media.licdn.com/dms/image/D4D03AQGLOAC...,Aya,...,"Licence, Technologies de l''information",May 21,May 21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,https://www.linkedin.com/in/amine-ounissi-839b...,2023-02-18T16:15:12.544Z,https://www.linkedin.com/in/amine-ounissi-839b...,NaN,Cloud Computing Engineering Student with a con...,Cloud & DevOps Engineering Student,"Paris, Île-de-France, France",https://media.licdn.com/dms/image/C4E03AQHTbnk...,Amine,...,"Associate's degree,Mathematics and Physics",February 2,February 2,CI/CD pipeline for a Java Spring-Boot microser...,Skills: Spring MVC · Kubernetes · ASP.NET · Cl...,github.com/Amine-22,NaN,NaN,NaN,NaN
4,NaN,https://www.linkedin.com/in/yasmine-dhib-8a904...,2023-02-18T16:15:52.792Z,https://www.linkedin.com/in/yasmine-dhib-8a904...,NaN,NaN,IT Architecture & Cloud Computing Engineer int...,"Tunis, Tunis, Tunisia",https://media.licdn.com/dms/image/D4E03AQHd01p...,Yasmine,...,"licence , Administration de / administrateur r...",NaN,NaN,Skills: Angular · DevOps · Kubernetes · Amazon...,NaN,NaN,"Paris, France",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,NaN,https://www.linkedin.com/in/cyrine-ben-hassine-,2023-02-18T17:05:56.358Z,https://www.linkedin.com/in/cyrine-ben-hassine-/,NaN,NaN,Software Engineer Intern @wevioo | Cloud Compu...,"Ariana Ville, Ariana, Tunisia",https://media.licdn.com/dms/image/C5603AQEzB4t...,Cyrine,...,NaN,NaN,NaN,conecption d'un systéme de gestion de configur...,Skills: .NET Core · GitHub · Jenkins · Ansible...,NaN,"Tunis, Tunisie",NaN,NaN,Skills: matlab
76,NaN,https://www.linkedin.com/in/iheb-sidhom-00474a190,2023-02-18T17:06:38.793Z,https://www.linkedin.com/in/iheb-sidhom-00474a...,NaN,NaN,Cloud operations engineer,"Paris, Île-de-France, France",https://media.licdn.com/dms/image/D5603AQHapxp...,Iheb,...,"Applied License, Telecommunication Technology/...",NaN,NaN,NaN,NaN,NaN,"Tunis, Tunisia",NaN,NaN,NaN
77,NaN,https://www.linkedin.com/in/amine-boujnah-1513...,2023-02-18T17:07:22.286Z,https://www.linkedin.com/in/amine-boujnah-1513...,NaN,"Hello there, I am Amine, a DevOps engineer wit...",AWS/DevOps/Cloud Engineer\n,"Sousse, Tunisia",https://media.licdn.com/dms/image/D4D03AQHN961...,Amine,...,Licence en STIC - sécurité des réseaux informa...,October 2,October 2,●\tCadrage du besoin et analyse de l’existant\...,NaN,NaN,NaN,NaN,NaN,NaN
78,NaN,https://www.linkedin.com/in/mohamed-h-06464b155,2023-02-18T17:08:05.166Z,https://www.linkedin.com/in/mohamed-h-06464b155/,NaN,NaN,DevOps & Cloud Engineer,"Tunis, Tunisia",https://media.licdn.com/dms/image/D4D03AQGUDdK...,Mohamed,...,NaN,April 29,April 29,SETTING UP A DEVOPS ORGANIZATION\nContainerize...,NaN,NaN,Tunisia,Development and automation project\n• Develop ...,NaN,NaN


,error,baseUrl,timestamp,linkedinProfile,email,description,headline,location,imgUrl,firstName,...,jobDescription2,jobLocation2,jobDateRange2,connectedOn,birthday,jobDescription,mail,phoneNumber,website,facebookUrl
0,Not a LinkedIn Profile URL,profileUrl,2023-02-20T11:28:56.927Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,https://www.linkedin.com/in/abassi-seifeddin-0...,2023-02-20T11:29:33.747Z,https://www.linkedin.com/in/abassi-seifeddin-0...,NaN,Currently in the third year of #CloudComputing...,Cloud et DevOps engineer,"Délégation Soliman, Gouvernorat Nabeul, Tunisie",https://media.licdn.com/dms/image/D4E03AQEILSG...,Abassi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,https://www.linkedin.com/in/mohamed-achraf-cho...,2023-02-20T11:30:13.662Z,https://www.linkedin.com/in/mohamed-achraf-cho...,NaN,"AWS Cloud , Microsoft Azure , Cloud Computing ...",Etudiant Ingénieur Informatique | Cloud Comput...,"Gouvernorat Tunis, Tunisie",https://media.licdn.com/dms/image/C4E03AQFPmwR...,Mohamed Achraf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,https://www.linkedin.com/in/ghada-laaribi-499a...,2023-02-20T11:30:52.457Z,https://www.linkedin.com/in/ghada-laaribi-499a...,NaN,NaN,IT Architecture and Cloud Computing Student @E...,"Gouvernorat Nabeul, Tunisie",NaN,Ghada,...,Migration d’un système d’information client ve...,Tunisie,juil. 2022 - sept. 2022 · 3 mois,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,https://www.linkedin.com/in/wael-belhadj-732a0...,2023-02-20T11:31:33.461Z,https://www.linkedin.com/in/wael-belhadj-732a0...,NaN,Etudiant ingénieur en cloud computing à ESPRIT...,Engineering Student in cloud computing at ESPRIT,"Tunis, Gouvernorat Tunis, Tunisie",https://media.licdn.com/dms/image/C4E03AQHRNWp...,Wael,...,NaN,"Tunis, Tunisie",juil. 2021 - sept. 2021 · 3 mois,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,NaN,https://www.linkedin.com/in/rafiknaccache,2023-02-20T12:22:12.524Z,https://www.linkedin.com/in/rafiknaccache/,NaN,Rafik Naccache is a software architect with ov...,"Senior Software Craftsman, Data engineer and C...",Grand Tunis Metropolitan Area,https://media.licdn.com/dms/image/C4D03AQEQ2sM...,Rafik,...,NaN,Tunis,juin 2022 - déc. 2022 · 7 mois,19 octobre,19 octobre,"Providing Cloud, Data and Architecture experti...",NaN,NaN,calendly.com/rafik-naccache/30min,NaN
74,NaN,https://www.linkedin.com/in/kasmi-lina,2023-02-20T12:22:51.868Z,https://www.linkedin.com/in/kasmi-lina/,NaN,NaN,Cloud Computing Engineering Student - ESPRIT,Plus de 500 relations,https://media.licdn.com/dms/image/D4D03AQFHJmd...,Kasmi,...,NaN,NaN,NaN,7 décembre,7 décembre,NaN,NaN,NaN,NaN,NaN
75,NaN,https://www.linkedin.com/in/amine-chibeni-5b26...,2023-02-20T12:23:32.740Z,https://www.linkedin.com/in/amine-chibeni-5b26...,NaN,Cloud computing engineering student looking fo...,DevOps & Cloud computing engineering student,"Gouvernorat Ariana, Tunisie",https://media.licdn.com/dms/image/C4E03AQGUiGm...,Amine,...,Mise en place d'une solution de scan des vulné...,NaN,févr. 2020 - juil. 2020 · 6 mois,NaN,NaN,Mise en place d'une application web pour la qu...,NaN,NaN,NaN,NaN
76,NaN,https://www.linkedin.com/in/guesmi-mohamed-208...,2023-02-20T12:24:15.587Z,https://www.linkedin.com/in/guesmi-mohamed-208...,NaN,"Linux.,Database skills,Programming,Networking,...",Cloud Computing & DevOps engineering student,"Tunis, Gouvernorat Tunis, Tunisie",https://media.licdn.com/dms/image/C4E03AQH0DTQ...,Guesmi,...,Compétences : Hadoop,"Ariana Governorate, Tunisia",déc. 2017 - aujourd’hui · 5 ans 3 mois,NaN,NaN,Compétences : Amazon EC2 · Amazon EKS · Terraf...,NaN,NaN,NaN,NaN


In [9]:
df2.shape

(80, 63)

In [11]:
df3.shape

(78, 63)

In [18]:
List=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]
dfP=pd.concat(List)

In [19]:
dfP.shape

(1198, 78)

In [20]:
dfP

,error,baseUrl,timestamp,linkedinProfile,email,headline,location,imgUrl,firstName,lastName,...,currentJob,job,url,name,query,category,additionalInfo,pastJob,summary,sharedConnections
0,Not a LinkedIn Profile URL,profileUrl,2023-02-16T22:04:31.406Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,https://www.linkedin.com/in/hamdi-ben-zaghden-...,2023-02-16T22:05:12.029Z,https://www.linkedin.com/in/hamdi-ben-zaghden-...,NaN,ERP - Business Intelligence Engineering Studen...,"Tunis, Tunisia",https://media.licdn.com/dms/image/D4E03AQHz0Jk...,Hamdi,Ben Zaghden,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,https://www.linkedin.com/in/essid-amine-912a86240,2023-02-16T22:05:52.374Z,https://www.linkedin.com/in/essid-amine-912a86...,NaN,Ingénieur Business Intelligence,500+ connections,https://media.licdn.com/dms/image/D4D03AQGxWc1...,Essid,Amine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,https://www.linkedin.com/in/hazem-baccouch-a66...,2023-02-16T22:06:33.285Z,https://www.linkedin.com/in/hazem-baccouch-a66...,NaN,Ingénieur en informatique spécialité Business ...,"Ben Arous, Tunisia",https://media.licdn.com/dms/image/C4E03AQFPwbz...,Hazem,Baccouch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,https://www.linkedin.com/in/youssef-dor%C3%A2a...,2023-02-16T22:07:13.234Z,https://www.linkedin.com/in/youssef-dor%C3%A2a...,NaN,Business Intelligence Student,"Ben Arous, Tunisia",https://media.licdn.com/dms/image/C4E03AQFTqqZ...,Youssef,Dorâa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,NaN,https://www.linkedin.com/in/narimen-hadjkacem-...,2023-02-20T12:24:36.849Z,https://www.linkedin.com/in/narimen-hadjkacem-...,NaN,Ingénieur informatique chez VERMEG for Insuran...,"Gouvernorat Tunis, Tunisie",https://media.licdn.com/dms/image/D4E03AQGWe9Z...,Narimen,Hadjkacem,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,Empty line,NaN,2023-02-20T12:24:41.067Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,Empty line,NaN,2023-02-20T12:24:41.069Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,Empty line,NaN,2023-02-20T12:24:41.071Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
dfP.to_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/TTDataScraping.csv",index=True)

### Data Activity 

In [24]:
dfActivity1=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/4DS1_Data.csv")
dfActivity2=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/4DS2_Data.csv")
#dfActivity3=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/4DS4_Data.csv")
dfActivity4=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/4DS6_Data.csv")
dfActivity5=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/4DS9_Data.csv")
dfActivity6=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/Data-4DS8.csv")
dfActivity7=pd.read_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/dataset_InterviewAlumni.csv")


In [25]:
ListActivity=[dfActivity1,dfActivity2,dfActivity4,dfActivity5,dfActivity6,dfActivity7]
dfPActivity=pd.concat(ListActivity)

In [26]:
dfPActivity.shape

(64, 63)

In [28]:
dfPActivity.to_csv("C:/Users/MSI/Desktop/Projet Data Science PI principal/Competency-based curriculum Platform-PI DATA SCIENCE/data/raw/DataActivityForm/TTDataActivity.csv",index=True)